In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col, monotonically_increasing_id, row_number
from pyspark.sql.functions import sum, col,avg,count,upper,round
import os

In [5]:
spark = SparkSession.builder \
    .appName("Reports Generation") \
    .getOrCreate()

In [65]:
print(os.getcwd())

output_dir = "output-reports"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

C:\akash_sahu\MiniProject


In [53]:
# Load the Dimension and Fact tables
customers_dim = spark.read.csv("output/customers_dim.csv", header=True, inferSchema=True)
products_dim = spark.read.csv("output/products_dim.csv", header=True, inferSchema=True)
sellers_dim = spark.read.csv("output/sellers_dim.csv", header=True, inferSchema=True)
time_dim = spark.read.csv("output/time_dim.csv", header=True, inferSchema=True)
transaction_dim = spark.read.csv("output/transaction_dim.csv", header=True, inferSchema=True)
inventory_fact = spark.read.csv("output/inventory_fact.csv", header=True, inferSchema=True)

In [54]:
# just checking 

customers_dim.show()
products_dim.show()
sellers_dim.show()
time_dim.show()
transaction_dim.show()
inventory_fact.show()

+---------------+-----------+------------------+-----------------+-----------------------+-------------+--------------+------------+-----------------+
|CUSTOMER_DIM_ID|CUSTOMER_ID|     Customer Name|CUSTOMER_LOGIN_ID|CUSTOMER_STREET_ADDRESS|CUSTOMER_CITY|CUSTOMER_STATE|CUSTOMER_ZIP|CUSTOMER_PHONE_NO|
+---------------+-----------+------------------+-----------------+-----------------------+-------------+--------------+------------+-----------------+
|              1|     100078|     Caldwell Wade|             5078|   216-4377 Semper, St.|     COLUMBUS|          OHIO|       94154|     027-779-9608|
|              2|     100003|       Fritz Grant|             5003|   Ap #897-7736 Eges...| PHILADELPHIA|  PENNSYLVANIA|       65342|     029-197-3614|
|              3|     100082|    Graiden Oneill|             5082|   P.O. Box 708, 751...|       BOSTON|MASSAACHUSETTS|       95485|     011-485-6145|
|              4|     100040|       Caryn Doyle|             5040|      3258 Massa Avenue|    

In [74]:
# transaction_dim.withColumn('sa', transaction_dim.PRODUCT_SELLING_PRICE).show()

# inventory_fact.withColumn('sa', inventory_fact.PRODUCT_SELLING_PRICE+2).show()



In [55]:


# Report 1: Transaction Amount Wise Top 10 Customers
top_10_customers = inventory_fact.join(customers_dim, "CUSTOMER_ID") \
    .join(time_dim, "TIME_DIM_ID") \
    .join(transaction_dim, "TRANSACTION_ID")\
    .where(col("YEAR") == 2011) \
    .groupBy("YEAR", "Customer Name",customers_dim.CUSTOMER_ID ) \
   .agg(sum("TRANSACTION_AMOUNT").alias("TOTAL_SELLING_PRICE")) \
    .orderBy("TOTAL_SELLING_PRICE",ascending = False) \
    .limit(10)


In [75]:
top_10_customers.show()

top_10_customers.toPandas().to_csv(f"{output_dir}/top_10_customers.csv")


+----+----------------+-----------+-------------------+
|YEAR|   Customer Name|CUSTOMER_ID|TOTAL_SELLING_PRICE|
+----+----------------+-----------+-------------------+
|2011|   Angelica Lamb|     100022|             176500|
|2011| Danielle Wilcox|     100083|             125450|
|2011|     Ifeoma Hale|     100017|             117300|
|2011|  Adena Carrillo|     100038|             106450|
|2011|Katelyn Garrison|     100059|             103550|
|2011|  Odette Patrick|     100011|             101000|
|2011| Debra Mcpherson|     100025|              89750|
|2011|  Claudia Carter|     100094|              89600|
|2011|     Echo Spence|     100079|              83600|
|2011|    Carly Parker|     100074|              82250|
+----+----------------+-----------+-------------------+



In [57]:
# Report 2: State wise number of orders
statewise_orders = inventory_fact.join(customers_dim, "CUSTOMER_ID") \
    .groupBy(upper(col("CUSTOMER_STATE")).alias("CUSTOMER_STATE")) \
    .agg(count("TRANSACTION_ID").alias("ORDER_COUNT")) \
    .orderBy(col("CUSTOMER_STATE"),ascending = True)

In [76]:
statewise_orders.show()

statewise_orders.toPandas().to_csv(f"{output_dir}/statewise_orders.csv")

+--------------------+-----------+
|      CUSTOMER_STATE|ORDER_COUNT|
+--------------------+-----------+
|             ARIZONA|         78|
|          CALIFORNIA|        295|
|DISTRICT OF COLUMBIA|         44|
|            ILLINOIS|         64|
|      MASSAACHUSETTS|         72|
|             NEWYORK|         51|
|                OHIO|        101|
|        PENNSYLVANIA|         62|
|               TEXAS|        233|
+--------------------+-----------+



In [59]:
# Report 3: Preferred mode of payment across states
payment_mode_by_state = inventory_fact.join(transaction_dim, "TRANSACTION_ID") \
    .groupBy("TRANSACTION_TYPE") \
    .agg(count("TRANSACTION_ID").alias("ORDER_COUNT")) \
    .orderBy(col("ORDER_COUNT") , ascending = True)


In [77]:
payment_mode_by_state.show()

payment_mode_by_state.toPandas().to_csv(f"{output_dir}/payment_mode_by_state.csv")

+----------------+-----------+
|TRANSACTION_TYPE|ORDER_COUNT|
+----------------+-----------+
|     CREDIT CARD|        108|
|      DEBIT CARD|        178|
|          PAYPAL|        199|
|             COD|        248|
|     NET BANKING|        267|
+----------------+-----------+



In [61]:
# Report 4: Quarterly profit for a particular year
# Profit = PRODUCT_SELLING_PRICE - PRODUCT_COST_PRICE

year_filter = 2011  #  desired year

# we are using group by "YEAR" because we want year in our output

quarterly_profit = inventory_fact.join(time_dim, "TIME_DIM_ID") \
    .filter(col("YEAR") == year_filter) \
    .groupBy("YEAR", "QUARTER") \
    .agg(sum(col("PRODUCT_SELLING_PRICE") - col("PRODUCT_COST_PRICE")).cast("int").alias("PROFIT")) \
    .orderBy("QUARTER")

In [78]:
quarterly_profit.show()

quarterly_profit.toPandas().to_csv(f"{output_dir}/quarterly_profit.csv")

+----+-------+------+
|YEAR|QUARTER|PROFIT|
+----+-------+------+
|2011|      1| 47030|
|2011|      2| 42941|
|2011|      3| 42572|
|2011|      4| 37421|
+----+-------+------+



In [ ]:
#updating the product table ( as we have multiple data for 

In [63]:
# Report 5: Quarterly sales count of each product category

# filtering for required year


year_filter = int(input("enter the year"))

quarterly_sales_count = inventory_fact.join(time_dim, "TIME_DIM_ID") \
    .join(products_dim, "PRODUCT_ID") \
    .filter(col("YEAR") == year_filter)\
    .filter(col("CATEGORY_ID") == 4005) \
    .groupBy("YEAR", "QUARTER", "CATEGORY_ID", "PRODUCT Name") \
    .agg(count("TRANSACTION_ID").alias("SALES_COUNT")) \
    .orderBy("YEAR", "QUARTER", "CATEGORY_ID")


enter the year 2011


In [79]:
quarterly_sales_count.show()

quarterly_sales_count.toPandas().to_csv(f"{output_dir}/quarterly_sales_count.csv")

+----+-------+-----------+------------+-----------+
|YEAR|QUARTER|CATEGORY_ID|PRODUCT Name|SALES_COUNT|
+----+-------+-----------+------------+-----------+
|2011|      1|       4005|      LAPTOP|         13|
|2011|      2|       4005|      LAPTOP|         13|
|2011|      3|       4005|      LAPTOP|         11|
|2011|      4|       4005|      LAPTOP|         13|
+----+-------+-----------+------------+-----------+



In [1]:
print("sdfa")

sdfa
